In [ ]:
import sys, os
sys.path.append("..")
sys.path.insert(0, os.path.expanduser("~/projects/bpack"))

In [ ]:
import pathlib
import itertools
from typing import NamedTuple, Set
from xml.etree import ElementTree as etree

In [ ]:
import bpack
import numpy as np

In [ ]:
import s1isp
from s1isp.udf import decode_ud
from s1isp.decoder import decode_stream
from s1isp.descriptors import ESesSignalType, PacketPrimaryHeader, PacketSecondaryHeader

In [ ]:
DATADIR = pathlib.Path(s1isp.__file__).parent.parent.joinpath("data").resolve()

raw_product = DATADIR / "S1B_S3_RAW__0SDV_20200615T162409_20200615T162435_022046_029D76_F3E6.SAFE"
raw_file = raw_product / "s1b-s3-raw-s-vv-20200615t162409-20200615t162435-022046-029d76.dat"

ref_product = DATADIR / "S1B_S3_RAW__0SDV_20200615T162409_20200615T162435_022046_029D76_F3E6.TIFF"
ref_file = ref_product / "S1B_S3_RAW__0SDV_20200615T162409_20200615T162435_022046_029D76_F3E6.TIFF_0003"
ref_header_file = ref_product / "S1B_S3_RAW__0SDV_20200615T162409_20200615T162435_022046_029D76_F3E6.TIFF_0003.xml"

In [ ]:
records, offsets, subcomm_data_records = decode_stream(raw_file)

In [ ]:
class Target(NamedTuple):
    packet_count: int
    label: str
    signal_types: Set[ESesSignalType]

In [ ]:
PRIMARY_HEADER_SIZE = bpack.calcsize(
    PacketPrimaryHeader,
    bpack.EBaseUnits.BYTES,
)
SECONDARY_HEADER_SIZE = bpack.calcsize(
    PacketSecondaryHeader,
    bpack.EBaseUnits.BYTES,
)

sizes = [PRIMARY_HEADER_SIZE + ph.packet_data_length + 1 for ph, _ in records]
offsets = list(itertools.accumulate([0] + sizes))

In [ ]:
def read_ref_data(headerfile, datafile, row, dtype=np.dtype("complex64")):
    xmldoc = etree.parse(headerfile)

    samples = int(xmldoc.findtext("./Channel/RasterInfo/Samples"))
    rowprefix = int(xmldoc.findtext("./Channel/RasterInfo/RowPrefixBytes"))

    linesize = rowprefix + samples * dtype.itemsize
    with open(datafile, "rb") as fd:
        fd.seek(row * linesize)
        rawdata = fd.read(linesize)
    
    header = rawdata[:rowprefix]
    data = np.frombuffer(rawdata[rowprefix:], dtype=dtype)

    return header, data

In [ ]:
targets = [
    Target(packet_count=0, label="noise", signal_types={ESesSignalType.noise}),
    Target(packet_count=8, label="txcal", signal_types={ESesSignalType.tx_cal}),
    Target(packet_count=408, label="echo", signal_types={ESesSignalType.echo}),
]

In [ ]:
with open(raw_file, "rb") as fd:
    for target in targets:
        ph, sh = records[target.packet_count]
        assert sh.counters_service.space_packet_count == target.packet_count
        assert sh.radar_configuration_support_service.ses_sbb_message.signal_type in target.signal_types

        offset = offsets[target.packet_count]
        size = sizes[target.packet_count]
        fd.seek(offset)
        isp_data = fd.read(size)

        tstmod = sh.fixed_ancillary_data_service.test_mode
        nq = sh.radar_sample_count_service.number_of_quads
        baqmod = sh.radar_configuration_support_service.baq_mode
        bindata = isp_data[PRIMARY_HEADER_SIZE + SECONDARY_HEADER_SIZE:]
        data = decode_ud(bindata, nq, baqmod, tstmod)

        filename = f"{target.packet_count:06d}-{target.label}.dat"
        with open(filename, "wb") as raw_fd:
            raw_fd.write(isp_data)

        filename = f"{target.packet_count:06d}-{target.label}.npz"
        if target.label == "echo":
            packet_offset = 408
            _, refdata = read_ref_data(ref_header_file, ref_file, target.packet_count - packet_offset)
            np.savez(filename, primary_header=ph, secondary_header=sh, udf=refdata, decoded_udf=data)
        else:
            np.savez(filename, primary_header=ph, secondary_header=sh, udf=data)